In [25]:
!pip install sqlalchemy

In [38]:
from sqlalchemy import create_engine, Column, Integer, String, Float, ForeignKey, DateTime, inspect, text
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import relationship, sessionmaker
from datetime import datetime

In [80]:
#Create tables(ORM approach)
from sqlalchemy import create_engine, Column, Integer, String, Float, ForeignKey
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import relationship
from sqlalchemy.exc import IntegrityError

# Define the Base class using declarative_base
Base = declarative_base()

# Define the User model
class User(Base):
    __tablename__ = 'users'

    id = Column(Integer, primary_key=True)
    username = Column(String(50), unique=True, nullable=False)
    email = Column(String(100), unique=True, nullable=False)

    orders = relationship('Order', back_populates='user')

    def __repr__(self):
        return f"<User(username='{self.username}', email='{self.email}')>"

# Define the Product model
class Product(Base):
    __tablename__ = 'Products'  # Table name changed to 'Products'

    id = Column(Integer, primary_key=True)
    name = Column(String(100), nullable=False)
    price = Column(Float, nullable=False)

    order_items = relationship('OrderItem', back_populates='product')

    def __repr__(self):
        return f"<Product(name='{self.name}', price={self.price})>"

# Define the Order model
class Order(Base):
    __tablename__ = 'orders'

    id = Column(Integer, primary_key=True)
    user_id = Column(Integer, ForeignKey('users.id'), nullable=False)

    user = relationship('User', back_populates='orders')
    order_items = relationship('OrderItem', back_populates='order')

    def __repr__(self):
        return f"<Order(id={self.id})>"

# Define the OrderItem model
class OrderItem(Base):
    __tablename__ = 'Order_items'  # Table name changed to 'Order_items'

    id = Column(Integer, primary_key=True)
    order_id = Column(Integer, ForeignKey('orders.id'), nullable=False)
    product_id = Column(Integer, ForeignKey('Products.id'), nullable=False)
    quantity = Column(Integer, nullable=False)

    order = relationship('Order', back_populates='order_items')
    product = relationship('Product', back_populates='order_items')

    def __repr__(self):
        return f"<OrderItem(order_id={self.order_id}, product_id={self.product_id}, quantity={self.quantity})>"

<ipython-input-80-032b6c8baac9>:8: MovedIn20Warning: The ``declarative_base()`` function is now available as sqlalchemy.orm.declarative_base(). (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base = declarative_base()


In [83]:
engine = create_engine('sqlite:///ecommerce_5.db', echo=True)

In [84]:
 with engine.connect() as connection:
      result = connection.execute(text("SELECT 1"))
      print(result.scalar())

2025-05-13 12:07:49,959 INFO sqlalchemy.engine.Engine BEGIN (implicit)


INFO:sqlalchemy.engine.Engine:BEGIN (implicit)


2025-05-13 12:07:49,961 INFO sqlalchemy.engine.Engine SELECT 1


INFO:sqlalchemy.engine.Engine:SELECT 1


2025-05-13 12:07:49,963 INFO sqlalchemy.engine.Engine [generated in 0.00361s] ()


INFO:sqlalchemy.engine.Engine:[generated in 0.00361s] ()


1
2025-05-13 12:07:49,966 INFO sqlalchemy.engine.Engine ROLLBACK


INFO:sqlalchemy.engine.Engine:ROLLBACK


In [93]:
# Create all tables defined in the Base metadata
Base.metadata.create_all(engine)

2025-05-13 12:13:18,178 INFO sqlalchemy.engine.Engine BEGIN (implicit)


INFO:sqlalchemy.engine.Engine:BEGIN (implicit)


2025-05-13 12:13:18,181 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("users")


INFO:sqlalchemy.engine.Engine:PRAGMA main.table_info("users")


2025-05-13 12:13:18,183 INFO sqlalchemy.engine.Engine [raw sql] ()


...

INFO:sqlalchemy.engine.Engine:PRAGMA main.table_info("Order_items")


2025-05-13 12:13:18,198 INFO sqlalchemy.engine.Engine [raw sql] ()


INFO:sqlalchemy.engine.Engine:[raw sql] ()


2025-05-13 12:13:18,200 INFO sqlalchemy.engine.Engine COMMIT


INFO:sqlalchemy.engine.Engine:COMMIT


In [94]:
# Use SQLAlchemy's inspect function to get table information
inspector = inspect(engine)

# Get a list of table names in the database
tables = inspector.get_table_names()

# Print the list of tables
print("Tables in the database:")
for table in tables:
    print(table)

2025-05-13 12:13:25,086 INFO sqlalchemy.engine.Engine BEGIN (implicit)


INFO:sqlalchemy.engine.Engine:BEGIN (implicit)


2025-05-13 12:13:25,088 INFO sqlalchemy.engine.Engine SELECT name FROM sqlite_master WHERE type='table' AND name NOT LIKE 'sqlite~_%' ESCAPE '~' ORDER BY name


INFO:sqlalchemy.engine.Engine:SELECT name FROM sqlite_master WHERE type='table' AND name NOT LIKE 'sqlite~_%' ESCAPE '~' ORDER BY name


2025-05-13 12:13:25,090 INFO sqlalchemy.engine.Engine [raw sql] ()


INFO:sqlalchemy.engine.Engine:[raw sql] ()


2025-05-13 12:13:25,093 INFO sqlalchemy.engine.Engine ROLLBACK


INFO:sqlalchemy.engine.Engine:ROLLBACK


Tables in the database:
Order_items
Products
orders
users


In [95]:
# Create a session to perform database operations
Session = sessionmaker(bind=engine)
session = Session()

In [96]:
#CRUD operations
#Can be improved with Transaction errors, for example by Try :... except Exception as e:

In [97]:
#User Table CRUD

# Create
def add_user(username, email):
  try:
    new_user = User(username=username, email=email)
    session.add(new_user)
    session.commit()
    return new_user.id
  except IntegrityError as e:
    session.rollback()
    print(f"Error: {e}")


# Read
def get_user(user_id):
    return session.query(User).filter(User.id == user_id).first()

# Update
def update_user(user_id, username=None, email=None):
    user = session.query(User).filter(User.id == user_id).first()
    if user:
        if username:
            user.username = username
        if email:
            user.email = email
        session.commit()

# Delete
def delete_user(user_id):
    user = session.query(User).filter(User.id == user_id).first()
    if user:
        session.delete(user)
        session.commit()

In [98]:
#Product Table CRUD

# Create
def add_product(name, price):
    new_product = Product(name=name, price=price)
    session.add(new_product)
    session.commit()
    return new_product.id

# Read
def get_product(product_id):
    return session.query(Product).filter(Product.id == product_id).first()

# Update
def update_product(product_id, name=None, price=None):
    product = session.query(Product).filter(Product.id == product_id).first()
    if product:
        if name:
            product.name = name
        if price is not None:  # Allow price to be set to 0
            product.price = price
        session.commit()

# Delete
def delete_product(product_id):
    product = session.query(Product).filter(Product.id == product_id).first()
    if product:
        for item in product.order_items:
            session.delete(item)
        session.delete(product)
        session.commit()

In [99]:
#Order Table CRUD

# Create
def add_order(user_id, order_items=None):
    new_order = Order(user_id=user_id)
    session.add(new_order)
    session.commit()
    if order_items:
        for item in order_items:
            add_order_item(new_order.id, item['product_id'], item['quantity'])
    return new_order.id

# Read
def get_order(order_id):
    return session.query(Order).filter(Order.id == order_id).first()

# Update
def update_order(order_id, user_id=None, order_items=None):
    order = session.query(Order).filter(Order.id == order_id).first()
    if order:
        if user_id:
            order.user_id = user_id
        if order_items is not None:
            for item in order_items:
                update_order_item(item['id'], item['product_id'], item['quantity'])
        session.commit()

# Delete
def delete_order(order_id):
    order = session.query(Order).filter(Order.id == order_id).first()
    if order:
        for item in order.order_items:
            session.delete(item)
        session.delete(order)
        session.commit()

In [100]:
#OrderItem Table CRUD

# Create
def add_order_item(order_id, product_id, quantity):
    new_order_item = OrderItem(order_id=order_id, product_id=product_id, quantity=quantity)
    session.add(new_order_item)
    session.commit()
    return new_order_item.id

# Read
def get_order_item(order_item_id):
    return session.query(OrderItem).filter(OrderItem.id == order_item_id).first()

# Update
def update_order_item(order_item_id, product_id=None, quantity=None):
    order_item = session.query(OrderItem).filter(OrderItem.id == order_item_id).first()
    if order_item:
        if product_id:
            order_item.product_id = product_id
        if quantity is not None:  # Allow quantity to be set to 0
            order_item.quantity = quantity
        session.commit()

# Delete
def delete_order_item(order_item_id):
    order_item = session.query(OrderItem).filter(OrderItem.id == order_item_id).first()
    if order_item:
        session.delete(order_item)
        session.commit()

In [101]:
def add_sample_data():
    # Adding users
    user_data = [
        ("Alice", "alice@example.com"),
        ("Bob", "bob@example.com"),
        ("Charlie", "charlie@example.com"),
        ("David", "david@example.com"),
        ("Eve", "eve@example.com"),
        ("Frank", "frank@example.com"),
        ("Grace", "grace@example.com"),
        ("Heidi", "heidi@example.com"),
        ("Ivan", "ivan@example.com"),
        ("Judy", "judy@example.com")
    ]
    for username, email in user_data:
        add_user(username, email)

    # Adding products
    product_data = [
        ("Laptop", 1000),
        ("Smartphone", 500),
        ("Tablet", 300),
        ("Monitor", 150),
        ("Keyboard", 50),
        ("Mouse", 25),
        ("Printer", 200),
        ("Router", 75),
        ("Webcam", 60),
        ("Headphones", 80)
    ]
    for name, price in product_data:
        add_product(name, price)

    # Adding orders
    for i in range(1, 11):
        # Here we assume that each user has one order for simplicity
        add_order(user_id=i)

    # Adding order_items
    order_item_data = [
        (1, 1, 2),
        (1, 2, 1),
        (2, 3, 1),
        (2, 4, 2),
        (3, 5, 3),
        (3, 6, 1),
        (4, 7, 2),
        (4, 8, 1),
        (5, 9, 1),
        (5, 10, 2),
        (6, 1, 1),
        (6, 2, 2),
        (7, 3, 1),
        (7, 4, 1),
        (8, 5, 2),
        (8, 6, 1),
        (9, 7, 3),
        (9, 8, 1),
        (10, 9, 1),
        (10, 10, 1)
    ]
    for order_id, product_id, quantity in order_item_data:
        add_order_item(order_id, product_id, quantity)

# Make sure to call the function to execute it
add_sample_data()

2025-05-13 12:13:37,329 INFO sqlalchemy.engine.Engine BEGIN (implicit)


INFO:sqlalchemy.engine.Engine:BEGIN (implicit)


2025-05-13 12:13:37,333 INFO sqlalchemy.engine.Engine INSERT INTO users (username, email) VALUES (?, ?)


INFO:sqlalchemy.engine.Engine:INSERT INTO users (username, email) VALUES (?, ?)


2025-05-13 12:13:37,336 INFO sqlalchemy.engine.Engine [generated in 0.00268s] ('Alice', 'alice@example.com')


...

INFO:sqlalchemy.engine.Engine:BEGIN (implicit)


2025-05-13 12:13:38,418 INFO sqlalchemy.engine.Engine SELECT "Order_items".id AS "Order_items_id", "Order_items".order_id AS "Order_items_order_id", "Order_items".product_id AS "Order_items_product_id", "Order_items".quantity AS "Order_items_quantity" 
FROM "Order_items" 
WHERE "Order_items".id = ?


INFO:sqlalchemy.engine.Engine:SELECT "Order_items".id AS "Order_items_id", "Order_items".order_id AS "Order_items_order_id", "Order_items".product_id AS "Order_items_product_id", "Order_items".quantity AS "Order_items_quantity" 
FROM "Order_items" 
WHERE "Order_items".id = ?


2025-05-13 12:13:38,419 INFO sqlalchemy.engine.Engine [cached since 0.4011s ago] (20,)


INFO:sqlalchemy.engine.Engine:[cached since 0.4011s ago] (20,)


In [102]:
# Convert the query result to a tuple of tuples for caching
def execute_sql_query(query):
    with engine.connect() as connection:
        result = connection.execute(text(query))
        if result.returns_rows:
            return tuple(tuple(row) for row in result)
        return ()

# Define a cached version of the function
@lru_cache(maxsize=32)
def cached_execute_sql_query(query):
    return execute_sql_query(query)

In [103]:
# Define a cached version of the function
from functools import lru_cache

@lru_cache(maxsize=32)
def cached_execute_sql_query(query):
    return execute_sql_query(query)

In [104]:
# Example usage

#Which products did user_1 order and in what quantities

sql_query = "SELECT user_id, product_id, quantity FROM users JOIN orders ON users.id=orders.id JOIN Order_items on orders.id=Order_items.order_id WHERE user_id=1"
results=cached_execute_sql_query(sql_query)

2025-05-13 12:13:43,288 INFO sqlalchemy.engine.Engine BEGIN (implicit)


INFO:sqlalchemy.engine.Engine:BEGIN (implicit)


2025-05-13 12:13:43,291 INFO sqlalchemy.engine.Engine SELECT user_id, product_id, quantity FROM users JOIN orders ON users.id=orders.id JOIN Order_items on orders.id=Order_items.order_id WHERE user_id=1


INFO:sqlalchemy.engine.Engine:SELECT user_id, product_id, quantity FROM users JOIN orders ON users.id=orders.id JOIN Order_items on orders.id=Order_items.order_id WHERE user_id=1


2025-05-13 12:13:43,292 INFO sqlalchemy.engine.Engine [generated in 0.00369s] ()


INFO:sqlalchemy.engine.Engine:[generated in 0.00369s] ()


2025-05-13 12:13:43,294 INFO sqlalchemy.engine.Engine ROLLBACK


INFO:sqlalchemy.engine.Engine:ROLLBACK


In [105]:
for row in results:
    print(row)

(1, 1, 2)
(1, 2, 1)


In [106]:
#Save the output log to a file on disk

import logging
from sqlalchemy import create_engine, text
from functools import lru_cache

# Configure logging
logging.basicConfig(
    filename='database_queries.log',  # Log file name
    filemode='w',  # Overwrite the log file each time the script runs; use 'a' for appending
    format='%(asctime)s - %(levelname)s - %(message)s',
    level=logging.INFO  # Set the logging level to INFO or DEBUG as needed
)

# Create an engine
engine = create_engine('sqlite:///ecommerce_5.db', echo=True)

# Convert the query result to a tuple of tuples for caching
def execute_sql_query(query):
    logging.info(f"Executing SQL query: {query}")
    with engine.connect() as connection:
        result = connection.execute(text(query))
        if result.returns_rows:
            rows = tuple(tuple(row) for row in result)
            logging.info(f"Query returned {len(rows)} rows.")
            return rows
        logging.info("Query returned no rows.")
        return ()

# Define a cached version of the function
@lru_cache(maxsize=32)
def cached_execute_sql_query(query):
    logging.info(f"Attempting to execute cached SQL query: {query}")
    return execute_sql_query(query)

# Example usage
sql_query = "SELECT * FROM users"
results = cached_execute_sql_query(sql_query)
for row in results:
    print(row)

2025-05-13 12:16:20,085 INFO sqlalchemy.engine.Engine BEGIN (implicit)


INFO:sqlalchemy.engine.Engine:BEGIN (implicit)


2025-05-13 12:16:20,088 INFO sqlalchemy.engine.Engine SELECT * FROM users


INFO:sqlalchemy.engine.Engine:SELECT * FROM users


2025-05-13 12:16:20,090 INFO sqlalchemy.engine.Engine [generated in 0.00437s] ()


INFO:sqlalchemy.engine.Engine:[generated in 0.00437s] ()


2025-05-13 12:16:20,093 INFO sqlalchemy.engine.Engine ROLLBACK


INFO:sqlalchemy.engine.Engine:ROLLBACK


(1, 'Alice', 'alice@example.com')
(2, 'Bob', 'bob@example.com')
(3, 'Charlie', 'charlie@example.com')
(4, 'David', 'david@example.com')
(5, 'Eve', 'eve@example.com')
(6, 'Frank', 'frank@example.com')
(7, 'Grace', 'grace@example.com')
(8, 'Heidi', 'heidi@example.com')
(9, 'Ivan', 'ivan@example.com')
(10, 'Judy', 'judy@example.com')


In [107]:
# Close the session
session.close()

2025-05-13 12:16:39,762 INFO sqlalchemy.engine.Engine ROLLBACK


INFO:sqlalchemy.engine.Engine:ROLLBACK
